In [2]:
import numpy as np
import matplotlib.pyplot as plt

from macti.PyNoxtli.geo.rectangle import Rectangle
from macti.PyNoxtli.fvm.sDiffusion import sDiffusion2D
from macti.PyNoxtli.fvm.pde import PDE
from macti.PyNoxtli.utils.displayInfo import printInfo
import macti.visual as mvis

def diffusion2D(Lx, Ly, Nx, Ny, TL, TR, TB, TT, k=1.0):
    #
    # Definición del dominio y condiciones de frontera
    placa = Rectangle(Lx, Ly)
    placa.boundaryConditions(dirichlet = {'LEFT':TL, 'RIGHT':TR, 'BOTTOM':TB, 'TOP':TT})
    #
    # Creación de la malla
    #
    malla     = placa.constructMesh(Nx, Ny) # Se construye la malla del dominio
    ivx, ivy, _ = malla.bounds(bi = 1, ei = Nx-1,
                               bj = 1, ej = Ny-1) # Grados de libertad
    nx  = malla.nx    # Número de nodos
    ny  = malla.ny    # Número de nodos
    nvx = malla.vx    # Número de volúmenes
    nvy = malla.vy    # Número de volúmenes
    dx  = malla.dx    # Tamaño de los volúmenes
    dy  = malla.dy    # Tamaño de los volúmenes
    #
    # Arreglo para almacenar la solución
    T = np.zeros((nvy+2, nvx+2)) # El arreglo contiene ceros
    T[-1,:] = TT      # Condición de frontera pared superior
    T[0,:] = TB      # Condición de frontera pared inferior
    T[:,0] = TL      # Condición de frontera pared izquierda
    T[:,-1] = TR      # Condición de frontera pared derecha
    #
    # Definición de la fuente 
    #
    Su = np.zeros((ivy, ivx)) # Por ahora no hay fuente
    #
    # Definición del esquema de disccretización
    #
    dif_scheme = sDiffusion2D(malla, Su, Gamma = k)
    #
    # Definición de la ecuación a resolver
    #
    laplace = PDE(placa, T)
    #
    # Creación del sistema lineal y su solución
    #
    Su.shape = ivy * ivx
    laplace.setNumericalScheme(dif_scheme)
    sol = laplace.solve()

    x, y, _ = malla.coordinatesMeshFVM()
    # Generación de una rejilla
    xg, yg = np.meshgrid(x, y, indexing='ij')
    
    vis = mvis.Plotter(1,3,[dict(aspect='equal'), dict(aspect='equal'), dict(aspect='equal')],
                      dict(figsize=(10,8)))
    
    vis.draw_domain(1, xg, yg)
    vis.plot_mesh2D(2, xg, yg, nodeson=True)
    vis.plot_frame(2, xg, yg)
    
    cax3 = vis.set_canvas(3,Lx,Ly)
    c = vis.contourf(3, xg, yg, T.T, levels=50, cmap='inferno')
    vis.fig.colorbar(c, cax=cax3, ticks = [T.min(), T.max()], shrink=0.5, orientation='vertical')
    vis.plot_mesh2D(3, xg, yg)
    vis.show()

ModuleNotFoundError: No module named 'macti.PyNoxtli.geo'

In [ ]:
import ipywidgets as wd

w_Lx = wd.FloatSlider(
    min=1.0, max=3.0, step=0.5, value=1.0,
    description='Lx',
    layout=wd.Layout(width='300px')
)

w_Ly = wd.FloatSlider(
    min=1.0, max=3.0, step=0.5, value=2.0,
    description='Ly',
    layout=wd.Layout(width='300px')
)

w_Nx = wd.IntSlider(
    min=5, max=10, step=1, value=5,
    description='Nx',
    layout=wd.Layout(width='300px')
)

w_Ny = wd.IntSlider(
    min=9, max=20, step=1, value=9,
    description='Ny',
    layout=wd.Layout(width='300px')
)

w_TL = wd.FloatSlider(
    min=0.0, max=2.0, step=0.5, value=0.0,
    description='Left',
    layout=wd.Layout(width='300px')
)

w_TR = wd.FloatSlider(
    min=0.0, max=2.0, step=0.5, value=0.0,
    description='Right',
    layout=wd.Layout(width='300px')
)
w_TB = wd.FloatSlider(
    min=0.0, max=2.0, step=0.5, value=0.0,
    description='Bottom',
    layout=wd.Layout(width='300px')
)

w_TT = wd.FloatSlider(
    min=0.0, max=2.0, step=0.5, value=1.0,
    description='Top',
    layout=wd.Layout(width='300px')
)

button = wd.Button(
    description="", icon='play',
    layout=wd.Layout(width='50px')    
)

ui = wd.VBox([wd.HBox([w_Lx, w_Ly, w_Nx, w_Ny]),
              wd.HBox([w_TL, w_TR, w_TB, w_TT]),
              wd.HBox([button])])

ui.layout = wd.Layout(border='solid 1px black')
ui.layout.width = '900px'

output = wd.Output()

display(ui, output)

def on_button_clicked(b):
    output.clear_output(wait=True)
    with output:
        diffusion2D(w_Lx.value, w_Ly.value, w_Nx.value, w_Ny.value,
                    w_TL.value, w_TR.value, w_TB.value, w_TT.value)

button.on_click(on_button_clicked)